In [1]:
import re

import numpy as np
import pandas as pd
import seaborn as sns

from datetime import datetime

In [2]:
df_sbuzz = pd.read_csv("syndata/Box Office/Recruitment x Box Office x Q1 2019 - Twitter.csv", sep="\t")
df_title = pd.read_csv("syndata/Box Office/Recruitment x Box Office x Q1 2019 - Titles Performace.csv", sep="\t")

In [3]:
df_sbuzz = df_sbuzz.drop(axis=1, labels=['Unnamed: 0'])
df_sbuzz['title'] = df_sbuzz['title'].str.lower()

df_title['Title'] = df_title['Title'].str.lower()

df_sbuzz['post_date'] = pd.to_datetime([datetime.strptime(x[0], '%Y-%m-%d').date() for x in list(df_sbuzz['post_date_time'].str.split('T'))])
df_sbuzz['post_time'] = [datetime.strptime(x[1].split('.')[0], '%H:%M:%S').time() for x in list(df_sbuzz['post_date_time'].str.split('T'))]

In [4]:
movie_dict = {}
c = 0

for i in df_sbuzz.index:
    if df_sbuzz.at[i, 'title'] not in movie_dict:
        c+=1
        movie_dict[df_sbuzz.at[i, 'title']] = c

for i in df_title.index:
    if df_title.at[i, 'Title'] not in movie_dict:
        c+=1
        movie_dict[df_title.at[i, 'Title']] = c

In [5]:
#movie_dict

In [6]:
# reformatting should be done better
for i in df_title.index:
    dates = df_title.at[i, 'Date']
    reg_date = re.sub("[.]|[ ]", "", dates)
    dt_list = []
    
    for d1 in reg_date.split('–'):   
        d1 = re.sub("Sept", "Sep", d1)
        m1 = ''
        if len(d1) > 3:
            month = m1.join(re.findall("[a-zA-Z]", d1))
            day = m1.join(re.findall("[0-9]", d1))

            s = '{} {} {}'.format(month, day, df_title.at[i, 'Year'])
            dt_list.append(datetime.strptime(s, '%b %d %Y').date())
            
        else:
            s = '{} {} {}'.format(month, d1, df_title.at[i, 'Year'])
            dt_list.append(datetime.strptime(s, '%b %d %Y').date())

    df_title.at[i, 'begin_date'] = dt_list[0]
    df_title.at[i, 'end_date'] = dt_list[1]

    new_range = []
    default_range = range(0, 7)
    wk_range = range(dt_list[0].weekday(), dt_list[1].weekday())
    
    if wk_range.start > wk_range.stop:
        k = wk_range.start
        for _ in range(len(default_range)):
            new_range.append(default_range[k % len(default_range)])
            k+=1
    else:
        new_range = list(wk_range)
    
    df_title.at[i, 'has_weekend'] = 5 in new_range or 6 in new_range
    df_title.at[i, 'nth_week_number'] = dt_list[0].isocalendar()[1]
    df_title.at[i, 'movie_id_t'] = int(movie_dict[df_title.at[i, 'Title']])
    

In [7]:
for i in df_sbuzz.index:
    df_sbuzz.at[i, 'movie_id_s'] = int(movie_dict[df_sbuzz.at[i, 'title']])
    df_sbuzz.at[i, 'nth_week_number'] = df_sbuzz.at[i, 'post_date'].isocalendar()[1]

In [8]:
#titles = list(df_title['Title'].unique())
#titles_buzz =  list(df_sbuzz['title'].unique())
#set(titles).intersection(titles_buzz)

In [9]:
df_title.sample(5) 

,Title,Date,Year,Rank,Weekend Gross,Gross-to-Date,Week # of release,begin_date,end_date,has_weekend,nth_week_number,movie_id_t
200,the meg,Aug. 30–Sept. 2,2018,2,"$898,690","$6,007,133",3,2018-08-30,2018-09-02,True,35.0,12.0
20,incredibles 2,Feb. 14–17,2019,47,$80,"$33,838,054",36,2019-02-14,2019-02-17,True,7.0,2.0
208,occupation,Jul. 26–29,2018,30,"$1,271","$3,046",3,2018-07-26,2018-07-29,True,30.0,21.0
51,the greatest showman,Apr. 12–15,2018,17,"$101,838","$26,344,069",16,2018-04-12,2018-04-15,True,15.0,3.0
160,overlord,Dec. 13–16,2018,13,"$105,029","$471,590",2,2018-12-13,2018-12-16,True,50.0,9.0


In [10]:
# df_title.loc[df_title['has_weekend'].isna()]
# df_title.loc[df_title['Weekend Gross'].isna()]
# df_title.loc[df_title['nth_week_number'].isna()]

In [11]:
# df_title.loc[df_title['Week # of release']==2]

In [12]:
df_sbuzz.sample(5)

,ID,title,post_link,post_date_time,post_content,user_name,user_bio,user_followers_count,user_followee_count,user_post_count,country,state,language,post_date,post_time,movie_id_s,nth_week_number
42685,8,coco,http://twitter.com/DustinTebbutt/status/932127...,2017-11-19T06:04:15.000+0000,Just missing the Coco Pops.. https://t.co/Ohv5...,DustinTebbutt,New Collab - Ok Moon - coming soon. head to: h...,2913,244,2402,AUS,New South Wales,en,2017-11-19,06:04:15,10.0,46.0
36610,5,wonder woman,http://twitter.com/derry715/status/86966902755...,2017-05-30T21:37:08.000+0000,RT @BIUK 'Wonder Woman' is one of the best sup...,derry715,Love life you really don't know how long you h...,961,2458,45506,AUS,Queensland,en,2017-05-30,21:37:08,7.0,22.0
15679,1,black panther,http://twitter.com/keels_rules/status/96582861...,2018-02-20T06:01:01.000+0000,RT @davidsocomedy I don’t know if it’s a sign ...,keels_rules,R.I.P Kim Jonghyun || Multifandom,12,57,267,AUS,Queensland,en,2018-02-20,06:01:01,4.0,8.0
69213,17,bright,http://twitter.com/AngelaMortimer2/status/9459...,2017-12-27T07:04:37.000+0000,RT @Sean_Marcano Bright is the type of movie i...,AngelaMortimer2,"I'm a painter & write Sci-fi, Crime and Fantas...",46554,46540,737583,AUS,NaN,en,2017-12-27,07:04:37,17.0,52.0
78639,25,lion,http://twitter.com/PPP_Plays/status/1096506663...,2019-02-15T20:28:55.000+0000,"Montaigne - For Your Love [Lion Arts Factory, ...",PPP_Plays,Tweeting each song played by @playpauseplay - ...,78,2,93891,AUS,South Australia,en,2019-02-15,20:28:55,25.0,7.0
